In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

#Descargar set de datos de MNIST (Numeros escritos a mano, etiquetados)
datos, metadatos = tfds.load('mnist', as_supervised=True, with_info=True)

#Obtenemos en variables separadas los datos de entrenamiento (60k) y pruebas (10k)
datos_entrenamiento, datos_pruebas = datos['train'], datos['test']

#Etiquetas de las 10 categorias posibles (simplemente son los numeros del 0 al 9)
nombres_clases = metadatos.features['label'].names

#Funcion de normalizacion para los datos (Pasar valor de los pixeles de 0-255 a 0-1)
#Hace que la red aprenda mejor y mas rapido
def normalizar(imagenes, etiquetas):
  imagenes = tf.cast(imagenes, tf.float32)
  imagenes /= 255 #Aqui lo pasa de 0-255 a 0-1
  return imagenes, etiquetas

#Normalizar los datos de entrenamiento y pruebas con la funcion que hicimos
datos_entrenamiento = datos_entrenamiento.map(normalizar)
datos_pruebas = datos_pruebas.map(normalizar)

#Agregar a cache (usar memoria en lugar de disco, entrenamiento mas rapido)
datos_entrenamiento = datos_entrenamiento.cache()
datos_pruebas = datos_pruebas.cache()

#Crear el modelo
modelo = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax) #Para redes de clasificacion
])

#Compilar el modelo
modelo.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

#Los numeros de datos en entrenamiento y pruebas (60k y 10k)
num_ej_entrenamiento = metadatos.splits["train"].num_examples
num_ej_pruebas = metadatos.splits["test"].num_examples

#El trabajo por lotes permite que entrenamientos con gran cantidad de datos se haga de manera mas eficiente
TAMANO_LOTE = 32

#Shuffle y repeat hacen que los datos esten mezclados de manera aleatoria para que la red
#no se vaya a aprender el orden de las cosas
datos_entrenamiento = datos_entrenamiento.repeat().shuffle(num_ej_entrenamiento).batch(TAMANO_LOTE)
datos_pruebas = datos_pruebas.batch(TAMANO_LOTE)

import math

#Entrenar
historial = modelo.fit(datos_entrenamiento, epochs=5, steps_per_epoch= math.ceil(num_ej_entrenamiento/TAMANO_LOTE))

d:\Codes\IA-Project\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  25%|██▌       | 1/4 [00:01<00:05,  1.71s/ url]

Dl Completed...:  75%|███████▌  | 3/4 [00:02<00:01,  1.71s/ url]

Dl Completed...: 100%|██████████| 4/4 [00:03<00:00,  1.11 url/s]


Dataset mnist downloaded and prepared to C:\Users\pms_l\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.
Epoch 1/5
1875/1875 [==============================] - 29s 14ms/step - loss: 0.2891 - accuracy: 0.9101
Epoch 2/5
1875/1875 [==============================] - 23s 12ms/step - loss: 0.1100 - accuracy: 0.9669
Epoch 3/5
1875/1875 [==============================] - 23s 13ms/step - loss: 0.0862 - accuracy: 0.9740
Epoch 4/5
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0721 - accuracy: 0.9790
Epoch 5/5
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0647 - accuracy: 0.9811


In [12]:
#Exportar el modelo en formato h5
modelo.save('model.h5')

d:\Codes\IA-Project\.env\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
#El equipo es Linux. Listemos el contenido de la carpeta actual para ver que se exporto el modelo
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
#Para convertirlo a tensorflow.js, primero debemos instalar la libreria
# !pip install tensorflowjs

In [5]:
#Crear carpeta donde se colocaran los archivos resultantes
!mkdir carpeta_out

In [14]:
#Realizar la exportacion a la carpeta de salida
!tensorflowjs_converter --input_format keras model.h5 carpeta_out

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\Codes\IA-Project\.env\Scripts\tensorflowjs_converter.exe\__main__.py", line 4, in <module>
  File "D:\Codes\IA-Project\.env\Lib\site-packages\tensorflowjs\__init__.py", line 21, in <module>
    from tensorflowjs import converters
  File "D:\Codes\IA-Project\.env\Lib\site-packages\tensorflowjs\converters\__init__.py", line 21, in <module>
    from tensorflowjs.converters.converter import convert
  File "D:\Codes\IA-Project\.env\Lib\site-packages\tensorflowjs\converters\converter.py", line 37, in <module>
    from tensorflowjs.converters import tf_saved_model_conversion_v2
  File "D:\Codes\IA-Project\.env\Lib\site-packages\tensorflowjs\converters\tf_saved_model_conversion_v2.py", line 42, in <module>
    import tensorflow_hub as hub
  File "D:\Codes\IA-Project\.env\Lib\site-packages\tensorflow_hub\__init__.py", line 93, in <module>
    fro

In [7]:
#Confirmar que en la carpeta de salida se hayan generado los archivos. Deben aparecer archivos "bin" y "json"
!ls carpeta_out

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
#Para descargarlos, da clic del lado izquierdo en el icono de la carpeta
#y expande carpeta_salida. En los archivos utiliza los 3 puntos para descargarlos